In [4]:
import os

SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

import json
import re
from ollama import chat

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            #response = client.chat.completions.create(
            #    model="deepseek-chat",
            #    messages=messages,
            #    tools=TOOLS_DEFINITION, # 告知模型可用的工具
            #    tool_choice="auto" # 允许模型自动决定是否使用工具
            #)
            response = chat(
                model="deepseek-r1:8b",
                messages=messages
                #tools=TOOLS_DEFINITION
            )
            response_message = response.message
            print(f"[模型生成结果] {response_message.content}")
            # --- START: 添加 JSON 提取和解析逻辑 ---
            #json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
            index = response_message.content.find("</think>")
            if index > 0:
                answer = response_message.content[index+8:]
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", answer, re.DOTALL)
                extracted_json_content = json_string_match.group(1)
                try:
                    final_response = json.loads(extracted_json_content)
                    print("Agent: 任务完成，成功解析最终JSON文案。")
                    return json.dumps(final_response, ensure_ascii=False, indent=2)
                except json.JSONDecodeError as e:
                    print(f"Agent: 提取到JSON块但解析失败: {e}")
                    print(f"尝试解析的字符串:\n{extracted_json_content}")
                    messages.append(response_message) # 解析失败，继续对话
            else:
                # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                try:
                    final_response = json.loads(response_message.content)
                    print("Agent: 任务完成，直接解析最终JSON文案。")
                    return json.dumps(final_response, ensure_ascii=False, indent=2)
                except json.JSONDecodeError:
                    print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                    messages.append(response_message) # 非JSON格式，继续对话
            # --- END: 添加 JSON 提取和解析逻辑 ---  
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
[模型生成结果] <think>
首先，用户要求我作为资深的小红书爆款文案专家来创作一个笔记文案。产品是“深海蓝藻保湿面膜”。需求包括：语气活泼甜美、包含标题和正文、至少5个相关标签和5个表情符号。

输出格式必须是JSON，并用markdown代码块包裹，例如：

```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```

我需要使用“Thought-Action-Observation”模式：先思考（collect information），然后行动（create the content），最后观察。

**Step 1: Thought (收集信息)**

作为专家，我必须理解产品卖点和小红书的风格。小红书笔记通常是个人、真实的分享，结合情感和实用性，吸引读者互动。

- **产品名称：深海蓝藻保湿面膜**
- **潜在卖点：**
  - 深海蓝藻是一种富含营养的成分，可能有抗氧化、保湿等功效。
  - 保湿面膜针对皮肤干燥问题。
  - 活泼甜美的语气意味着要使用亲切、热情的语言，比如感叹号、昵称化表达。

用户指定至少5个相关标签和表情符号。标签应该与产品主题相关，如#美容、#护肤等。表情符号需要活泼甜美，所以选择可爱、兴奋的表情。

收集更多信息：
- 深海蓝藻的特性：可能包括天然成分、深层保湿、改善肤质。
- 爆款文案的特点：高互动（鼓励评论、分享）、高转化（促使购买）。
- 语气：活泼甜美 – 使用感叹号、波浪线、亲昵称呼，如“宝贝们”或“亲爱的”。

结构化思考：
- **标题**：需要吸引眼球，包含关键词如“深海蓝藻”、“保湿面膜”，并用表情符号增强吸引力。例如：“深海蓝藻保湿面膜！超级补水的神仙礼物✨”
- **正文**：分成引人入胜的开头、产品描述、个人体验和号召性结尾。
  - 开头：引起共鸣，比如分享自己的皮肤问题。
  - 主体：解释产品好处，使用卖点。
  - 结尾：鼓励行动，如购买或尝试。

确保至少5个标签：